# Releasing your package

Once you have [implemented a bunch of function, classes, and methods](02-Package-Structure.ipynb#Package-layout) in your package and made sure your package can be [installed in the standard way](02-Package-Structure.ipynb#Installing-your-code), once you have set up [version control with git](03-git-and-GitHub.ipynb#Version-control) and you are using [GitHub to develop your code online](03-git-and-GitHub.ipynb#Using-GitHub-to-build-a-community-for-your-code), once you have written [documentation](04-Documentation.ipynb#Basics-of-good-documentation) explaining how to [install and use your code](04-Documentation.ipynb#Using-sphinx-to-write-and-generate-documentation-for-your-package), you have created an [API for all of the functionality in your package](04-Documentation.ipynb#Including-docstrings-into-the-sphinx-documentation) and you are [hosting your package's documentation online](04-Documentation.ipynb#Automatically-building-and-hosting-your-documentation-on-readthedocs.io), once you have written a [comprehensive test suite](05-Tests.ipynb#Testing-your-code) that checks that [your code works as you expect](05-Tests.ipynb#Writing-simple-tests), this test suite [covers a large fraction of your package](05-Tests.ipynb#Test-coverage) and you are using [continuous integration to run your test suite](06-Continuous-Integration.ipynb#Automatically-building-and-testing-your-code:-continuous-integration) automatically upon every change that you make to your code, you are ready to release a first version of your package to the world! 

Many scientists still release their code by creating a source distribution (a "tarball") and linking to it on their website, but this fails to take advantage of the large amount of support in the Python community (and beyond) to make releasing different versions of your code easy and allowing your code to be installed in standard ways on different platforms with minimal headaches for the users of your package. The primary venue for releasing Python packages is the Python Packge Index (``PyPI``) and it is releasing your package on ``PyPI`` that I focus on in this chapter. An additional popular method for releasing your code, especially if it contains compiled code and/or dependencies that are difficult to install, is to make your package installable by Anaconda's ``conda install`` command. The most common way to do this for scientific packages is to use [conda-forge](https://conda-forge.org/), but describing how to get your package onto ``conda-forge`` is beyond the scope of these notes.

## Versioning your code

Before you release a first version of your code, you should decide on how you want to label different versions of your code, because once you release the first version, you'll want to start work on the next version! When we first created the [setup.py file](02-Package-Structure.ipynb#The-setup.py-file), we included a version string, but I will now discuss versioning in a little more detail.

Your code's version could be any unique string that states the version (e.g., "v1"), but to make your code's version easy to interpret for a wide variety of users and compatible with standard Python functions to process version numbers, your code should follow the [standard Python versioning scheme](https://www.python.org/dev/peps/pep-0440/). Python version strings should follow the format
```
[N!]N(.N)*[{a|b|rc}N][.postN][.devN]
```
which I will briefly unpack here. We'll ignore the ``[N!]`` part, the [version epoch](https://www.python.org/dev/peps/pep-0440/#version-epochs), which is only used when packages change the way they version and which the author of these notes has never seen in practice.

For released versions, you only have to pay attention to the
```
N(.N)*
```
part of the version string. This states that any release can be any string of integers separated by periods (e.g., "0.1.0"). While arbitrary lengths are in principle allowed, most packages use either two integers separated by a period (e.g., "1.0") or three integers, (e.g., "2.1.3"). Upon a release, either the last integer is increased by 1 or one of the earlier integers in the sequence is increased and all following integers are set to 0. So, for example, version "0.1" could be increased to either "0.2" or to "1.0" in the next release, and version "1.3.2" could be increased to either of "2.0.0", "1.4.0", or "1.3.3". Note that the version string does *not* contain "v", that is, the version is "1.0.0", not "v1.0.0" (that can be easy to forget!).

Standard Python usage of version numbers is *not* to use semantic versioning as advocated by [semver](https://semver.org/), but to use a more loosely defined structure of "major.minor" or "major.minor.patch" where the "major" version is increased only when a package's structure changes dramatically (causing much backwards incompatibility, e.g., Python 2 to Python 3) or when the package matures significantly to deserve a "1.x.x" version (having had "0.x.x" until then; e.g., [scipy's relatively recent version](https://docs.scipy.org/doc/scipy/reference/release.1.0.0.html#why-1-0-now) "1.0.0", 16 years after the first release). Therefore, for most Python packages, major version changes are rare (but some packages update the major version more often, e.g., [astropy](https://docs.astropy.org/en/stable/whatsnew/)). Typical releases will therefore increase the minor version number, e.g., going from version "1.1" to "1.2". The third integer, called "patch" above, indicates small changes such as bug fixes or updates because of dependency changes. Such patch releases would typically be made from a branch of the code created at the latest minor version release to track bug fixes and other small changes, while development of the next minor version continues in the main development branch. 

The remainder of the possible version string
```
[{a|b|rc}N][.postN][.devN]
```
deals with pre- and post-release updates to a package. The ``[{a|b|rc}N]`` part indicates pre-release versions, with ``a`` indicating an alpha release, ``b`` a beta release, and ``rc`` a "release candidate". the ``N`` that can optionally follow each of these indicates the N-th version (e.g., "a2" would be the second alpha release). Unless you are developing a package with many users and/or that provides essential infrastructure where each release needs to be thoroughly tested, you will typically not make use of these versions (but you could if you want). The ``[.postN]`` is reserved for changes to your code that happen "post release". These are changes, not to the source of your code itself, but to the distribution of your code. You can for example create a ``.post1`` release if you forgot to include the License file in your release; if you simply want to add the file, you should create a new release (because a release, once made, is final), but you are not changing any of the code. Finally, the ``[.devN]`` is used to indicate development versions of your code. For example, "1.1.dev" is the development version that will lead to release "1.1" eventually. Note that there are a bunch of [alternative syntaxes](https://www.python.org/dev/peps/pep-0440/#normalization) allowed by the standard (for example, you can use "alpha" instead of "a"), but for new versions it's best to follow the actual standard discussed here.

One of the advantages of all packages using this standard versioning scheme is that this can be easily parsed by automated parsers, to check, for example, which version of a dependency users have installed on their systems. One way to do this is to use ``setuptools``' [pkg_resources](https://setuptools.readthedocs.io/en/latest/pkg_resources.html), which has a function ``parse_version`` (documented [here](https://setuptools.readthedocs.io/en/latest/pkg_resources.html#parsing-utilities), that returns an object that represents the version and that can be compared to other such objects. For example, we can check that the installed version of ``scipy`` is at least later than version "0.19" by doing
```
from pkg_resources import parse_version
import scipy
print(parse_version(scipy.__version__) >= parse_version('0.19'))
```
``parse_version`` correctly deals with development, pre-release, and post-release versions.

You should include your package's version in the ``setup.py`` file and define the ``__version__`` attribute in your package's top-level ``__init__.py`` file (``exampy/__init__.py`` in the example package); you will likely also want to define it in the documentation's ``source/conf.py``. Because your package's version then appears in multiple places in your code, you need to remember to update it everywhere when you increment the version number. One way to do this is to make it a part of your release checklist (discussed below), you can also make use of automated tools like [bumpversion](https://github.com/peritus/bumpversion), which can be configured to automatically update the version string in multiple files whenever you invoke ``bumpversion``. 

## Preparing for your package's release

In preparation for your package's initial and each subsequent release, you'll want to add/update some files to/in your repository. It's useful to list files that need to be updated every release as part of a *release checklist* (which you can add as a ``RELEASE_CHECKLIST.md`` file in your repository as well) that explicitly lists all of the steps involved in releasing each version of your package, from preparing the release, over uploading your package to PyPI, to setting up the next development version. 

Some of the files that you will want to add or update for every release are:

* A ``HISTORY.txt`` or ``HISTORY.md`` (for formatted GitHub content) file that lists major changes and additions to your package since the last release. For the first release, this file can simply state that this is the first release, but at each subsequent release you should update this file with all of the major changes since the previous release. It's easiest to keep this file up to date by adding to it as you develop each version, rather than writing at the end. If you find yourself having to write it at the end, go through all Issues and Pull Requests to find major changes, or look through the ``git`` log from the last release (although if you are making the recommended frequent commits, this will be a long and detailed log). One way to help yourself keep track of major changes is to submit Pull Requests for all of these even if you are making the changes yourself and you could just merge them directly.

  If your documentation includes a page summarizing major changes, you'll want to update that one as well.
  
* Update the version number to that of the release you are going to make, typically at least in the ``setup.py`` file and the top level ``__init__.py`` file. Typically, you should have been using a version "x.y.z.dev" while developing version "x.y.z" and in that case this is as simple as removing the ".dev" part of the version string.

* To obtain fine-grained control over which files are included in the source distribution, you need to use a ``MANIFEST.in`` file. Many files are included by default, most importantly: all Python files included in your package (through the ``packages`` keyword of the ``setuptools.setup`` command in your ``setup.py`` file), the ``setup.py`` file itself, any files specified in ``setuptools.setup``'s ``package_data`` keyword, scripts included in the setup, C source files for any C extensions specified in ``setuptools.setup``, various ``README`` formats (without extension, or with ``.txt``, ``.rst``, or ``.md`` extension), and the ``MANIFEST.in`` file itself. Test files following ``test/test*.py`` are also included (not ``tests/test*.py``). The ``MANIFEST.in`` file then gives you the option to include extra files or to exclude files from the standard list. To include a file, add a line ``include FILE`` in ``MANIFEST.in`` where ``FILE`` is the path to the file, which can include standard wildcards. To exclude a file, add a line ``exclude FILE`` (e.g., to exclude the tests, add ``exclude test/test*.py``). One file you'll want to include is the LICENSE file. If you have other useful files in your repository, e.g., ``INSTALL.md`` with installation instructions (typically given in the README file) or ``AUTHORS.md`` with contributors to your code, you can also include them in the ``MANIFEST.in`` file. If your package includes a C extension, you'll have to include the header files, because they will not by default be copied to the source distribution.

  As an example, for the ``exampy`` example package, I created the following ``MANIFEST.in``
  ```
  include LICENSE README.md
  exclude tests/test*.py
  ```
  to include the ``LICENSE`` file, make sure to include the ``README.md`` file (it should be included anyway, but it does not hurt to make sure), and to make sure to exclude the tests.

  If you have added files to your package since the last release, you might have to add some of them to ``MANIFEST.in`` to make sure that they are included in the source distribution. You can get a list of all files added since your last release by running
  ```
  git diff --name-status PREV_RELEASE_HASH | grep ^A
  ```
  where ``PREV_RELEASE_HASH`` is the ``git`` hash corresponding to the previous release.
  
Once you're happy with all of the necessary additions and changes, you should make sure to have commit all of the changes to your repository and push them to GitHub such that your automated tests can run one last time to check that all is well.

Provided that your tests pass, you can then move on to the next step, which is tagging the next release and creating it on GitHub. To create a tag, simply run
```
git tag x.y.z
```
where ``x.y.z``is the version of the release. Push this to GitHub with
```
git push --tags
```
Then you can go to your package's GitHub page and navigate to the "Releases" tab, in the row above the directory structure of your repository:

When you have pushed the tag, you will see that GitHub has created a release, that currently just looks like a stub:

You can create the actual release by clicking on this, clicking on "Edit tag", and giving the release a title and short description (as usual, using MarkDown formatting), and hitting "Publish release" on the page that looks like this:

Then you have created the release of your code on GitHub!

## Uploading your package to the Python Package Index (``PyPI``)

Mention wheels. Should I figure out how to build them with appveyor and upload?